In [ ]:
import sympy as sp
x,y,z,t = sp.symbols('x y z t')
def hessian(f):
    return sp.Matrix([[f.diff(x).diff(x),f.diff(x).diff(y),f.diff(x).diff(z)],
                      [f.diff(y).diff(x),f.diff(y).diff(y),f.diff(y).diff(z)],
                      [f.diff(z).diff(x),f.diff(z).diff(y),f.diff(z).diff(z)]])
def devgrad(f):
    Gf = sp.Matrix([[f[0].diff(x),f[0].diff(y),f[0].diff(z)],
                    [f[1].diff(x),f[1].diff(y),f[1].diff(z)],
                    [f[2].diff(x),f[2].diff(y),f[2].diff(z)]])
    return Gf - Gf.trace()/3*sp.Matrix.eye(3)
def curl(f):
    Cf = sp.Matrix([[f[0,2].diff(y)-f[0,1].diff(z),f[0,0].diff(z)-f[0,2].diff(x),f[0,1].diff(x)-f[0,0].diff(y)],
                    [f[1,2].diff(y)-f[1,1].diff(z),f[1,0].diff(z)-f[1,2].diff(x),f[1,1].diff(x)-f[1,0].diff(y)],
                    [f[2,2].diff(y)-f[2,1].diff(z),f[2,0].diff(z)-f[2,2].diff(x),f[2,1].diff(x)-f[2,0].diff(y)]])
    return Cf
def symcurl(f):
    Cf = curl(f)
    return (Cf+Cf.transpose())/2
def div(f):
    return sp.Matrix([f[0,0].diff(x)+f[0,1].diff(y)+f[0,2].diff(z), 
                      f[1,0].diff(x)+f[1,1].diff(y)+f[1,2].diff(z),
                      f[2,0].diff(x)+f[2,1].diff(y)+f[2,2].diff(z)])
def divdiv(f):
    return (f[0,0].diff(x).diff(x) +
            f[1,0].diff(y).diff(x) +
            f[2,0].diff(z).diff(x) +
            f[0,1].diff(x).diff(y) +
            f[1,1].diff(y).diff(y) +
            f[2,1].diff(z).diff(y) +
            f[0,2].diff(x).diff(z) +
            f[1,2].diff(y).diff(z) +
            f[2,2].diff(z).diff(z) )
def simplifyMat(A):
    for i in range(A.rows):
        for j in range(A.cols):
            A[i,j] = A[i,j].simplify()
    return A
def ev(f,ax,ay,az):
    return f.subs(x,ax).subs(y,ay).subs(z,az)
def S(A):
    SA = A.transpose() - A.trace()*sp.Matrix.eye(3)
    return simplifyMat(SA)
def dt(A):
    At = sp.Matrix.eye(3)
    for i in range(A.rows):
        for j in range(A.cols):
            At[i,j] = A[i,j].diff(t)
    return At
def L2(A,B):
    return sp.integrate(sp.integrate(sp.integrate(
        (sp.hadamard_product(A,B)*sp.Matrix.ones(A.cols,A.rows)).trace(),
        (x,0,1)),(y,0,1)),(z,0,1))

In [ ]:
import re
def cppcode(f):
    s = sp.ccode(f.simplify())
    s = re.sub("sin","autodiff::detail::sin",s)
    s = re.sub("cos","autodiff::detail::cos",s)
    s = re.sub("exp","autodiff::detail::exp",s)
    s = re.sub("pow","autodiff::detail::pow",s)
    return s
ADB = "      return autodiff1st(\n        [](autodiff::real x, autodiff::real y, autodiff::real z, autodiff::real t)->autodiff::real {\n          return "
ADE = ";\n        },X,t,dx,dy,dz);\n"
def ADPrint(A):
    if (A.cols == 1):
        print("    if (i == 0) {\n",ADB,cppcode(A[0]),ADE,
              "    } else if (i == 1) {\n",ADB,cppcode(A[1]),ADE,
              "    } else {\n",ADB,cppcode(A[2]),ADE,
              "    };",sep='')
    else:
        print("    if (i == 0 && j == 0) {\n",ADB,cppcode(A[0,0]),ADE,
              "    } else if (i == 0 && j == 1) {\n",ADB,cppcode(A[0,1]),ADE,
              "    } else if (i == 0 && j == 2) {\n",ADB,cppcode(A[0,2]),ADE,
              "    } else if (i == 1 && j == 0) {\n",ADB,cppcode(A[1,0]),ADE,
              "    } else if (i == 1 && j == 1) {\n",ADB,cppcode(A[1,1]),ADE,
              "    } else if (i == 1 && j == 2) {\n",ADB,cppcode(A[1,2]),ADE,
              "    } else if (i == 2 && j == 0) {\n",ADB,cppcode(A[2,0]),ADE,
              "    } else if (i == 2 && j == 1) {\n",ADB,cppcode(A[2,1]),ADE,
              "    } else {\n",ADB,cppcode(A[2,2]),ADE,
              "    };",sep='')


In [ ]:
def print0(f):
    df = devgrad(f)
    print("  double f0(unsigned i, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(f)
    print("  }\n  double df0(unsigned i, unsigned j, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(df)
    print("  }")
def print1(f):
    df = symcurl(f)
    print("  double f1(unsigned i, unsigned j, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(f)
    print("  }\n  double df1(unsigned i, unsigned j, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(df)
    print("  }")
def print2(f):
    df = divdiv(f)
    print("  double f2(unsigned i, unsigned j, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(f)
    print("  }\n  double df2(const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    print(ADB,cppcode(df),ADE,sep='')
    print("  }")

In [ ]:
l1,l2,c = sp.symbols('lambda_1 lambda_2 c')
P1C0 = sp.Matrix([[0,l1,0],[-l2,0,0],[0,0,0]])*sp.cos(c*z-c*t)
P2C0 = S(sp.Matrix([[(l1-l2)/2,0,0],[0,(l2-l1)/2,0],[0,0,-(l1+l2)/2]])*sp.cos(c*z-c*t))
P1C1 = sp.Matrix([[0,0,-l1],[0,0,-l1],[0,0,0]])*sp.cos(c*x+c*y-c*t)
P2C1 = S(sp.Matrix([[l1,0,0],[0,-l1,0],[0,0,0]])*sp.cos(c*x+c*y-c*t))

In [ ]:
def checkSol(A,G):
    print(simplifyMat(A.diff(t) + curl(G)) == sp.Matrix.zeros(3,3))
    print(simplifyMat(G.diff(t) - symcurl(A)) == sp.Matrix.zeros(3,3))
    print(simplifyMat(div(A)) == sp.Matrix.zeros(3,1))
    print(divdiv(G).simplify() == 0)

In [ ]:
checkSol(P1C0,P2C0)
checkSol(P1C1,P2C1)

In [ ]:
print("struct Sol0 {")
print("  double t;")
print("  static constexpr double c = std::acos(-1.), lambda_1 = 2., lambda_2 = 1.;")
print1(P1C0)
print2(P2C0)
print("};")
print("struct Sol1 {")
print("  double t;")
print("  static constexpr double c = std::acos(-1.), lambda_1 = 2., lambda_2 = 1.;")
print1(P1C1)
print2(P2C1)
print("};")